In [18]:
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
import anywidget
import traitlets
import jupyter
from tweet_browser_test import tweet_browser as tb
import voila
from matplotlib import pyplot as plt
from IPython.display import display
import pandas as pd
import io
import math

TWEETS_PER_PAGE = 20

out = widgets.Output()

def startSession(file):
    if file['type'] == 'xls':
        df = pd.read_excel(io.BytesIO(file.content))
    else:
        df = pd.read_csv(io.BytesIO(file.content))
    db = tb.DataBaseSim(df)
    s = tb.Session(db)
    browser = Browser(s, out)
    

def selectColumns (row, colHeaders: list):
    result = []
    for j in colHeaders:
        result.append(row[s.headerDict[j]])
    return result

class SearchBar(anywidget.AnyWidget):
    _esm = "anywidget/searchBar.js"
    _css = "anywidget/searchBar.css"
    value = traitlets.List([]).tag(sync=True)
    header = traitlets.Unicode("").tag(sync=True)
    placeholder = traitlets.Unicode("").tag(sync=True)
    count = traitlets.Int(0).tag(sync=True)
    
class TweetDisplay(anywidget.AnyWidget):
    _esm = "anywidget/tweetDisplay.js"
    _css = "anywidget/tweetDisplay.css"
    value = traitlets.List([]).tag(sync=True)
    
class DatasetDisplay(anywidget.AnyWidget):
    _esm = "anywidget/datasetDisplayPart.js"
    _css = "anywidget/misc.css"
    size = traitlets.Int().tag(sync=True)
    fileName = traitlets.Unicode().tag(sync=True)
    
class PageSelect(anywidget.AnyWidget):
    _esm = "anywidget/pageSelect.js"
    _css = "anywidget/pageSelect.css"
    tweetsPerPage = traitlets.Int(TWEETS_PER_PAGE).tag(sync=True)
    value = traitlets.CInt(1).tag(sync=True)
    totalTweets = traitlets.Int().tag(sync=True)
    changeSignal = traitlets.Int(0).tag(sync=True)
    
# File upload
fileUp = widgets.widgets.FileUpload(
    accept='.csv, .txt, .xls, .tsv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    description='Change Dataset'
)
fileUp.add_class("change-input")
datasetDisplayCustom = DatasetDisplay()
datasetDisplayCustom.add_class("text-block")
fileUpBar = widgets.Box([datasetDisplayCustom, fileUp])
fileUpBar.add_class("dataset-display")

class SortBar(anywidget.AnyWidget):
    _esm = "anywidget/sortBar.js"
    _css = "anywidget/sortBar.css"
    sortScope = traitlets.Unicode("Displayed Examples").tag(sync=True)
    sortColumn = traitlets.Unicode("None").tag(sync=True)
    sortOrder = traitlets.Unicode("DESC").tag(sync=True)

class ToggleSwitch(anywidget.AnyWidget):
    _esm = "anywidget/toggleSwitch.js"
    _css = "anywidget/toggleSwitch.css"
    value = traitlets.Int(0).tag(sync=True)
    label = traitlets.Unicode("").tag(sync=True)

class Browser:
    def __init__(self, s, out):
        self.s = s
        self.mainScreen = True
        self.colHeaders = list(s.headerDict.keys())
        self.createWidgets()
        self.resetDisplay()
        
    def resetDisplay(self, b = None):
        out.clear_output(True)
        if self.mainScreen == True:
            with out:
                #display(self.mustInclude)
                display(self.optionsBar)
                display(self.searchButton)
                display(self.advancedButton)
                display(self.tweetDisplay)
        else:
            with out:
                display(self.advancedPage)
                display(self.tempButton)

            
    def searchKeyword(self, b):
#         while(self.s.currentSet.size < self.s.length):
#             self.s.back()
        self.s.currentSet = self.s.base

        if(self.exclude.count > 0):
            self.s.exclude(self.exclude.value)
        
        if(self.mustInclude.count > 0):
            self.s.searchKeyword(self.mustInclude.value, False)
        
        if(self.containOneOf.count > 0):
            self.s.searchKeyword(self.containOneOf.value, True)
            
        for i in range(self.geography.count):
            self.s.filterBy("State", self.geography.value[i])
            
        for i in range(self.userName.count):
            self.s.filterBy("SenderScreenName", self.userName.value[i])
            
        if(self.fromDate.value != None and self.toDate.value != None):
            self.s.filterDate(self.fromDate.value.strftime('%Y-%m-%d'), self.toDate.value.strftime('%Y-%m-%d'))
        
        if(self.s.currentSet.size < self.s.length):
            self.getTweets(b)
    
    
    def getTweets(self, b):
        #self.resetDisplay()
        dataSet = self.s.getCurrentSubset()
        tempArr = []
        
        pageNum = self.pageSelect.value
        
        self.pageSelect.totalTweets = len(dataSet)
        self.pageSelect.changeSignal += 1
        
        sorted = self.getSortedTweets(pageNum)
        
        for i in range(len(sorted)):            
            tempArr.append(sorted.iloc[i].to_json())
        
        self.tweetDisplay.value = tempArr
    
    def getSortedTweets(self, pageNum):
        ans = self.s.getCurrentSubset()
            
        if self.sortBar.sortColumn == "None":
            return ans.iloc[(pageNum-1) * TWEETS_PER_PAGE : min(pageNum * TWEETS_PER_PAGE, len(ans))]

        asc = True
        na_pos = "first"
        column = self.sortBar.sortColumn
        keyFunc = None
            
        if (column == "Username" or column == "SenderScreenName"):
            keyFunc = userNameToLower

        if (self.sortBar.sortOrder == "DESC"):
            asc = False
            na_pos = "last"
        if (self.sortBar.sortScope == "Entire Dataset"):
            ans = ans.sort_values(by=[column], ascending=asc, na_position = na_pos, key=keyFunc)
            ans = ans.iloc[(pageNum-1) * TWEETS_PER_PAGE : min(pageNum * TWEETS_PER_PAGE, len(ans))]
        elif (self.sortBar.sortScope == "Displayed Examples"):
            sortedPage = ans.iloc[(pageNum-1) * TWEETS_PER_PAGE : min(pageNum * TWEETS_PER_PAGE, len(ans))]
            sortedPage = sortedPage.sort_values(by=[column], ascending=asc, na_position = na_pos, key=keyFunc)
            ans = sortedPage
        else:
            with out:
                print("An Error has occured while sorting")
        
        return ans
        
    def createWidgets(self):
        self.searchButton = widgets.Button(description='Search')
        self.searchButton.on_click(self.searchKeyword)
        self.tweetDisplay = TweetDisplay()
        self.datasetDisplay = fileUpBar
        self.pageSelect = PageSelect()
        self.pageSelect.observe(self.updateTweets, names=["value"])
        self.generateSummary = widgets.Button(description="Generate AI Summary")
        self.generateSummary.add_class("generic-button")
        self.sortBar = SortBar()
        self.sortBar.observe(self.updateTweets, names=["sortScope", "sortColumn", "sortOrder"])
        self.optionsBar = widgets.Box(children = [self.datasetDisplay, self.pageSelect, self.generateSummary, self.sortBar])
        self.optionsBar.layout = widgets.Layout(align_items = "center", justify_content = "space-between")
        self.advancedButton = widgets.Button(description="Search & Filter")
        self.advancedButton.on_click(self.openSearchMenu)
        self.keyWordSearch = widgets.HTML(value = "<b>Keyword Search<b/>")
        self.keyWordSearch.add_class("keyword-search")
        self.mustInclude = SearchBar(header = "Must Include", placeholder='e.g "civil liberty" means the result must contain the words "civil" and "liberty"')
        self.containOneOf = SearchBar(header = "Contain One Of", placeholder='e.g. "census result" means the result may contain either "census" or "result"')
        self.exclude = SearchBar(header = "Exclude", placeholder='e.g. "toxic" means the result will not contain the word "toxic"')
        self.searches = widgets.VBox([self.keyWordSearch, self.mustInclude, self.containOneOf, self.exclude])
        self.searches.add_class("search-box")
        self.tempButton = widgets.Button(description = 'Close')
        self.tempButton.on_click(self.closeSearchMenu)
        self.filterBy = widgets.HTML(value = "<b>Filter By<b/>")
        self.dateRange = widgets.HTML(value = "<b style='font-size: 1.17em;'>Date Range <b/>")
        self.fromDate = widgets.DatePicker(description = "From")
        self.toDate = widgets.DatePicker(description = "To")
        self.dateBox = widgets.VBox([self.dateRange, self.fromDate, self.toDate])
        self.allowRetweets = ToggleSwitch(label = "Retweets")
        self.geography = SearchBar(header = "Geography", placeholder = "Search")
        self.userName = SearchBar(header = "Username", placeholder = "Search")
        self.filterBox = widgets.VBox([self.filterBy, self.dateBox, self.allowRetweets, self.geography, self.userName])
        self.filterBox.add_class("filter-box")
        self.advancedPage = widgets.HBox([self.searches, self.filterBox])
        # self.advancedPage.layout = widgets.Layout(justify_content = 'space-between')
    
    def updateTweets(self, change):
        self.getTweets(None)
    
    def openSearchMenu(self, change):
        self.mainScreen = False
        self.resetDisplay()
        
    def closeSearchMenu(self, change):
        self.mainScreen = True
        self.resetDisplay()
    
def fileHandler(change):
    startSession(fileUp.value[0])
    
def userNameToLower(input):
    return input.str.lower()

        
with out:
    display(fileUpBar)


fileUp.observe(fileHandler, names=["value"])

out

Output()